In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
import warnings
warnings.filterwarnings('ignore')

# all import statements
import numpy as np
import pandas as pd
import pydicom as pyd
import os
import matplotlib.pyplot as plt
import mudicom

from os.path import dirname, join
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
from PIL import Image

from scipy.misc import imresize
from scipy.signal import convolve2d
import scipy
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

from numpy import newaxis

from skimage.measure import shannon_entropy
from skimage import io, color, img_as_ubyte
from skimage.feature import greycomatrix, greycoprops
from sklearn.metrics.cluster import entropy


import math
from skimage.measure import label, regionprops

from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb

import matplotlib.patches as mpatches

import statistics

from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max

import cv2
from skimage import img_as_float
from skimage import exposure

In [2]:
import pickle

from tqdm import tqdm
import os
import sys

In [3]:
def load_pickle(fileName):
    with open(fileName, "rb") as fp:
        file = pickle.load(fp)
    return file 

def write_pickle(file, fileName):
    with open(fileName, "wb") as fp:
        pickle.dump(file, fp)

### GLCM

In [4]:

def s_entropy(dcm):
    return shannon_entropy(dcm)

def entropy_simple(dcm):
    return entropy(dcm)

class glcm:
    def __init__(self, image):
        distance = [1, 2, 3]
        angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
        self.image = img_as_ubyte(image.astype('int64'))
        self.glcm_mat = greycomatrix(self.image, distances = distance, angles = angles, symmetric = True, normed = True)
        self.properties = ['correlation', 'homogeneity', 'contrast', 'energy']
            
    def correlation(self):
        return greycoprops(self.glcm_mat, 'correlation').flatten()
    
    def homogeneity(self):
        return greycoprops(self.glcm_mat, 'homogeneity').flatten()
    
    def contrast(self):
        return greycoprops(self.glcm_mat, 'contrast').flatten()
    
    def energy(self):
        return greycoprops(self.glcm_mat, 'energy').flatten()
    
    def glcm_all(self):
        return np.hstack([greycoprops(self.glcm_mat, props).ravel() for props in self.properties])
    

### Region Props

In [5]:
# thresh_ada = threshold_otsu(image_adahist)
# bw_ada = closing(image_adahist > thresh_ada, square(3))
# bw_clear_ada = clear_border(bw_ada)

class region_props:
    def __init__(self, image):
        self.image = image
        self.thresh = threshold_otsu(self.image)
        self.bw = closing(self.image > self.thresh, square(3))
        self.bw_clear = clear_border(self.bw)
        self.bw_label = label(self.bw_clear)
        self.regions = regionprops(self.bw_label)
#         self.idx = 0
        
    def plot_image_bw(self):
        plt.imshow(self.bw, cmap = plt.cm.bone)
    
    def plot_image(self):
        plt.imshow(self.bw_clear, cmap=plt.cm.bone)
        
    def plot_image_with_labels(self):
        plt.imshow(self.bw_label, cmap=plt.cm.bone)
        
    def max_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            idx = lista.index(max(lista))
            return max(lista)
        else:
            return -1
    
    def eccentricity(self):
        
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            idx = lista.index(max(lista))
            return self.regions[idx].eccentricity
        else:
            return -1
    
    def euler_number(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            idx = lista.index(max(lista))
            return self.regions[idx].euler_number
        else:
            return -1
    
    def solidity(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            idx = lista.index(max(lista))
            return self.regions[idx].solidity
        else:
            return -1
    
    def perimeter(self):
        lista = []
        for e in self.regions:
            lista.append(e.perimeter)
        if len(lista)>=1:
            return max(lista)
        else:
            return -1
    
    def mean_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>=1:
            return statistics.mean(lista)
        else:
            return -1
    
    def std_area(self):
        lista = []
        for e in self.regions:
            lista.append(e.area)
        if len(lista)>2:
            return statistics.stdev(lista)
        else:
            return -1
    
    def thresh_img(self):
        return self.thresh
        

### Moments

In [6]:
class moments:
    
    def __init__(self, image):
        self.image = image
        self.moment = cv2.moments(self.image)
        self.hu = cv2.HuMoments(self.moment)
        
    def get_moments(self):
#         keys = [key for key in self.moment.keys()]
        values = [value for value in self.moment.values()]
        return values
    
    def get_HuMoments(self):
        moments_hu = []
        for m in range(len(self.hu)):
            moments_hu.append(self.hu[m][0])
        return moments_hu

#### DataFrame dcm_1

In [7]:
dcm_t = load_pickle("dcm-test")
pat_t = load_pickle("pat-test")

In [8]:
len(dcm_t), len(pat_t)

(3000, 3000)

In [9]:
data_dcm_t = dict()
for dc in tqdm(range(len(dcm_t))):
    data_dcm_t[pat_t[dc]] = [s_entropy(dcm_t[dc]), entropy_simple(dcm_t[dc]), moments(dcm_t[dc]).get_moments(), moments(dcm_t[dc]).get_HuMoments(),
                      region_props(dcm_t[dc]).max_area(), region_props(dcm_t[dc]).eccentricity(), region_props(dcm_t[dc]).euler_number(),
                      region_props(dcm_t[dc]).solidity(), region_props(dcm_t[dc]).perimeter(), region_props(dcm_t[dc]).mean_area(), 
                      region_props(dcm_t[dc]).std_area(), region_props(dcm_t[dc]).thresh_img(), glcm(dcm_t[dc]).correlation(), glcm(dcm_t[dc]).homogeneity(),
                      glcm(dcm_t[dc]).contrast(), glcm(dcm_t[dc]).energy()]

100%|██████████| 3000/3000 [39:28<00:00,  1.25it/s]


In [10]:
df_dcm_t = pd.DataFrame(data_dcm_t).T

In [11]:
df_dcm_t.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
27c4fce5-77d2-4b4f-b314-c97b6523ff94,7.58424,5.25699,"[104491092.0, 56624839001.0, 64752980775.0, 37...","[0.0012856731357998989, 1.0604881141657069e-08...",10833,0.837174,-8,0.679228,1979.37,119,805.052,111,"[0.994073354437144, 0.9885400631047354, 0.9933...","[0.5129934949203385, 0.44636240607808736, 0.54...","[44.548391869806935, 86.00175914857589, 49.907...","[0.048288425830066296, 0.044190376804432094, 0..."
223e7914-1cdb-4fe4-a086-0969efed0ee5,7.77717,5.39073,"[125066030.0, 60748299973.0, 70032920821.0, 38...","[0.0012749845661348541, 2.5702436919531166e-08...",289,0.853282,1,0.661327,95.3198,11,40.6079,115,"[0.9989033654976042, 0.9980303183555193, 0.998...","[0.43747770850684103, 0.3684341593948492, 0.46...","[8.928389235092865, 16.016343550919274, 9.3452...","[0.03464417213734621, 0.03238136505250272, 0.0..."
297ef567-0d06-473e-831e-6fe0698a2d66,7.62273,5.28367,"[111976788.0, 54195063327.0, 63117513066.0, 32...","[0.0012140272965091578, 4.618755633523459e-08,...",1498,0.963778,0,0.320428,413.12,80,243.029,108,"[0.9943221025464034, 0.9893021696463543, 0.994...","[0.5228808811281439, 0.46229877824746235, 0.58...","[38.51696335838222, 72.43922528663803, 39.4268...","[0.03359900904368914, 0.030380252953336788, 0...."
0f1dc462-55cf-4c50-9ffe-16e1cec3cb79,6.81288,4.72233,"[106819243.0, 52489271426.0, 58167884344.0, 33...","[0.0014335055409899489, 9.536202338184548e-09,...",725,0.973575,-2,0.489534,318.652,14,54.2125,90,"[0.9979558610276797, 0.9956665569707506, 0.997...","[0.5704886977700285, 0.48896470853187457, 0.58...","[6.898810751160804, 14.61843102293391, 9.43270...","[0.049996611238771846, 0.0443781788437396, 0.0..."
27eef917-370c-41a9-b97a-07fd4cf3fb33,7.23248,5.01317,"[111320446.0, 61636907033.0, 63732361332.0, 41...","[0.0012523692847897405, 1.711887635191213e-08,...",939,0.973575,-3,0.736471,340.439,37,148.149,77,"[0.9919164213807479, 0.9867753143536582, 0.993...","[0.6099285323579047, 0.5400111558448983, 0.651...","[44.33788012432794, 72.44934349645347, 36.5302...","[0.05365613574919529, 0.04929011473675117, 0.0..."


In [12]:
df_dcm_t.to_pickle('df_dcm_t')

In [13]:
df_dcm_t.to_csv('df_dcm_t.csv')